# Binary Classification Deep Learning Model for [Project Name] Using Keras Version 2
### David Lowe
### October 4, 2019
Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: [Sample Paragraph - The purpose of this project is to construct a predictive model using various machine learning algorithms and to document the end-to-end steps using a template. The Pima Indians Diabetes dataset is a binary classification situation where we are trying to predict one of the two possible outcomes.]

INTRODUCTION: [Sample Paragraph - This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.]

ANALYSIS: [Sample Paragraph - The baseline performance of the model achieved an accuracy score of 77.47%. Using the optimized tuning parameter available, the model processed the validation dataset with an accuracy of ??.??%, which was worst than the training data and possibly due to over-fitting]

CONCLUSION: [Sample Paragraph - For this dataset, the model built using Keras and TensorFlow achieved a satisfactory result and should be considered for future modeling activities]

Dataset Used: [Pima Indians Diabetes Data Set]

Dataset ML Model: Binary classification with numerical attributes

Dataset Reference: [https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv]

One potential source of performance benchmarks: [https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/]

Any deep-learning modeling project genrally can be broken down into about seven major tasks:
0. Prepare Environment
1. Load Data
2. Define Model
3. Compile Model
4. Fit Model
5. Evaluate Model
6. Finalize Model

# Section 0. Prepare Environment

In [1]:
# Create the random seed numbers for reproducible results
seedNum = 88

In [2]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import os
import smtplib
from datetime import datetime
from email.message import EmailMessage
from sklearn.model_selection import train_test_split

In [3]:
# Configure a new global `tensorflow` session
import keras as K
from keras.layers import Dense
import tensorflow as tf
tf.set_random_seed(seedNum)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.backend.set_session(sess)

ModuleNotFoundError: No module named 'keras'

In [ ]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set the scoring metric
scoring = 'accuracy'

In [ ]:
# Set up the email notification function
def email_notify(msg_text):
    sender = os.environ.get('MAIL_SENDER')
    receiver = os.environ.get('MAIL_RECEIVER')
    gateway = os.environ.get('SMTP_GATEWAY')
    smtpuser = os.environ.get('SMTP_USERNAME')
    password = os.environ.get('SMTP_PASSWORD')
    if sender==None or receiver==None or gateway==None or smtpuser==None or password==None:
        sys.exit("Incomplete email setup info. Script Processing Aborted!!!")
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from Keras Binary Classification Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP(gateway, 587)
    server.starttls()
    server.login(smtpuser, password)
    server.send_message(msg)
    server.quit()

In [ ]:
if (notifyStatus): email_notify("Phase 0 Prepare Environment completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 1. Load Data

In [ ]:
if (notifyStatus): email_notify("Phase 1 Load Data has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [ ]:
# Load the dataset
dataset = np.loadtxt('pima-indians-diabetes.data.csv', delimiter=',')
print(dataset)

In [ ]:
# Split the original dataset into input (X) and output (y) variables
X_original = dataset[:,0:8]
y_original = dataset[:,8]

In [ ]:
# Split the data further into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X_original, y_original, test_size=0.25, random_seed=seedNum)

In [ ]:
if (notifyStatus): email_notify("Phase 1 Load Data completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 2. Define Model

In [ ]:
if (notifyStatus): email_notify("Phase 2 Define Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [ ]:
# Define the Keras model
model = K.models.Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
if (notifyStatus): email_notify("Phase 2 Define Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 3. Compile Model

In [ ]:
if (notifyStatus): email_notify("Phase 3 Compile Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [ ]:
# Compile the Keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[scoring])

In [ ]:
if (notifyStatus): email_notify("Phase 3 Compile Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 4. Fit Model

In [ ]:
if (notifyStatus): email_notify("Phase 4 Fit Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [ ]:
# Fit the Keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=1)

In [ ]:
if (notifyStatus): email_notify("Phase 4 Fit Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 5. Evaluate Model

In [ ]:
if (notifyStatus): email_notify("Phase 5 Evaluate Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [ ]:
# Evaluate the Keras model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))

In [ ]:
if (notifyStatus): email_notify("Phase 5 Evaluate Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 6. Finalize Model

In [ ]:
if (notifyStatus): email_notify("Phase 6 Finalize Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [ ]:
# Make class predictions with the model
predictions = model.predict_classes(X_original)

# Summarize the first 20 cases
for i in range(20):
	print('%s => %d (expected %d)' % (X_original[i].tolist(), predictions[i], y_original[i]))

In [ ]:
if (notifyStatus): email_notify("Phase 6 Finalize Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [ ]:
print ('Total time for the script:',(datetime.now() - startTimeScript))